<a href="https://colab.research.google.com/github/seokhee516/kaggle-practice/blob/main/Binary%20classification/Porto%20Seguro%E2%80%99s%20Safe%20Driver%20Prediction/XGBoost%20CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gcsfs

     |████████████████████████████████| 123 kB 6.0 MB/s 
     |████████████████████████████████| 1.3 MB 43.8 MB/s 
     |████████████████████████████████| 142 kB 65.7 MB/s 
     |████████████████████████████████| 294 kB 73.9 MB/s 


In [2]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50

In [3]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc

In [4]:
# Compute gini
@jit
def eval_gini(y_true, y_prob):
  y_true = np.asanyarray(y_true)
  y_true = y_true[np.argsort(y_prob)]
  ntrue = 0
  gini = 0
  delta = 0
  n = len(y_true)
  for i in range(n-1, -1, -1):
    y_i = y_true[i]
    ntrue += y_i
    gini += y_i, dela
    delta += 1 - y_i
  gini = 1 - 2 *gini /(ntrue * (n - ntrue))
  return gini

In [5]:
def gini_xgb(preds, dtrain):
  labels = dtrain.get_label()
  gini_score = -eval_gini(labels, preds)
  return [('gini', gini_score)]

In [6]:
def add_noise(series, noise_level):
  return series * (1 + noise_level *np.random.randn(len(series)))

In [7]:
def target_encode(trn_series=None,
                  val_series=None,
                  tst_series=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
  assert len(trn_series) == (target)
  assert trn_series.name == tst_series.name
  temp = pd.concat([trn_series, target], axis=1)
  averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean","count"])
  smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf)))
  prior = target.mean()
  averages[target.name] = prior * (1-smoothing) + averages["mean"] * smoothing
  averages.drop(["mean", "count"], axis=1, inplace = True)
  ft_trn_series  = pd.merge(
      trn_series.to_frame(trn_series.name),
      averages.reset_index().rename(columns={
          'index':target.name, targget.name:'averages'
      }),
      on = trn_series.name,
      how ='left')['average'].rename(trn_series.name + '_mean').filna(prior)
  ft_val_serieis.index = trn_series.index
  ft_val_serieis = pe. merge(
      val_series.to_frame(val_series.name),
      averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
      on=val_series.name,
      how='left'
  )['average'].rename(trn_series.name + '_mean').fillna(prior)
  ft_tst_series.index = tst_series.test_index
  return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [8]:
# Road data
GCS_DS_PATH = "gs://kds-e5602443d7485cc7c65ae998ed306d9f26534ba34b6cda928f9be8b0"
train_df = pd.read_csv(GCS_DS_PATH+'/train.csv', na_values="-1")
test_df = pd.read_csv(GCS_DS_PATH+"/test.csv", na_values="-1")

In [9]:
# from olivier
train_features = [
  "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]
# add combinations
combs = [
         ('ps_reg_01', 'ps_car_02_cat'),
         ('ps_reg_01', 'ps_car_04_cat'),
]

In [10]:
# Process data
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

In [11]:
# 복습 필요
start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)

In [12]:
X = train_df[train_features]
test_df = test_df[train_features]
f_cats = [f for f in X.columns if "_cat" in f]

In [13]:
y_valid_pred = 0*y
y_test_pred = 0

In [14]:
# set up folds
K = 5
kf = KFold(n_splits = K, random_state=1, shuffle= True)
np.random.seed(0)

In [15]:
# set up classifier
model = XGBClassifier(
    n_estimators = MAX_ROUNDS,
    max_depth = 4,
    objective='binary:logistic',
    learning_rate = LEARNING_RATE,
    subsample=.8,
    min_child_weight=6,
    colsample_bytree=.8,
    scale_pos_weight=1.6,
    gamma=10,
    reg_alpha=8,
    reg_lambda=1.3
)

In [16]:
# Run CV
for i, (train_index, test_index) in enumerate(kf.split(train_df)):
  y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
  X_train, X_valid = X.iloc[train_index, :].copy(), X.iloc[test_index, :]. copy()
  X_test = test_df.copy()
  print("\nFold ", i)
  # Encode data
    for f in f_cats:
      X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f+ "_avg"] = target_encode(
          trn_series = X_train[f], 
          val_series = X_valid[f], 
          tst_series = X_test[f],
          target = y_train, 
          min_samples_leaf = 200, 
          smoothing =10, 
          noise_level = 0
      )
  # Run model for this fold
  if OPTIMIZE_ROUNDS:
    eval_set = [(X_valid, y_valid)]
    fit_model = model.fit(X_train, y_train,
                          eval_set = eval_set,
                          eval_metric = gini_xgb,
                          early_stopping_rounds = EARLY_STOPPING_ROUNDS,
                          verbose = False)
    print("Best N trees = ", model.best_ntree_limit)
    prin(" Best gini = ", model.best_score)
  else:
    fit_model = model.fit(X_train, y_train)
  # Generate validation predictions for this fold
  pred = fit_model.predict_prob(X_valid)[:,1]
  print("Gini = ", eval_gini(y_valid, pred))
  y_valid_pred.iloc[test_index] = pred

  # Accumulate test set predictions
  y_test_pred += fit_model.predict_proba(X_test)[:,1]
  
  del X_test, X_train, X_valid, y_train

y_test_pred /= K
print("\nGini for full training set:")
eval_gini(y,y_valid_pred)

IndentationError: ignored